## Taller Neo4j
El objetivo de este taller es embarrarse un poco las manos con Cypher y utitilizar el driver de Neo4j para Python.
Vamos a:
* Usar Cypher y Python para CRUD nodos y relaciones.
* Hacer unos ejercicios en Cypher y Python.

Este notebook le debe service de tutorial y de clade práctica.

In [16]:
from py2neo import Graph, Schema, authenticate

authenticate("neo4j:7474", "neo4j", "qwerty")
graph = Graph("http://neo4j:7474/db/data/")
graph.neo4j_version

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:5: DeprecationWarning: Graph.neo4j_version is deprecated, use DBMS.kernel_version instead


(3, 0, 7)

### Cypher
Vamos a tomar la asistencia. Con la siguiente consulta en Cypher creamos dos nodos Persona, "Olemis" y "Kinane" además de un nodo Taller "Neo4j Python" con una relación "Asistió".

In [17]:
query = """
CREATE (:Persona{nombre:"Kinane"})-[:ASISTIÓ]->(taller:Taller{título: "Neo4j Python"}),
(:Persona{nombre:"Olemis"})-[:ASISTIÓ]->(taller);
"""

data = graph.run(query)

Vamos a recuperar el nodo Taller y contar la asistencia

In [18]:
query = """
MATCH (taller:Taller)
WHERE taller.título = "Neo4j Python"
MATCH (taller)<-[:ASISTIÓ]-(inscritos:Persona)
RETURN count(inscritos) as inscritos;
"""

data = graph.run(query)

for d in data:
    print(d)

('inscritos': 2)


Marque su asistencia, creando una relación similar entre un nuevo nodo Persona y el Taller.

In [19]:
# Aquí

Ahora que has podido crear tu primer nodo y relación en Cypher, intenta añdir nuevos nodos y relaciones. 

In [20]:
query = """
MATCH (taller:Taller),
(kinane:Persona{nombre: "Kinane"})
WHERE taller.título = "Neo4j Python"
CREATE (taller)-[:PRESENTADO_POR]->(cuba:Meetup {título: "Cubantech"}),
(kinane)-[:ANFITRIÓN]->(beirut:Meetup {título: "Graph Databases Beirut"}),
(cuba)-[:COLABORÓ_CON]->(beirut),
(beirut)-[:COLABORÓ_CON]->(cuba)
"""

data = graph.run(query)

Por ejemplo con entidades de direción postal, interés, profesión, etc.

In [21]:
# Aquí

Vamos a añadir unas restricciones de títulos de los talleres

In [22]:
query = """
CREATE CONSTRAINT ON (taller:Taller) ASSERT taller.título IS UNIQUE
"""

data = graph.run(query)

Escribe una consulta cuyo resultado sea las demás personas que asistieron a este taller

In [23]:
# Aquí

### py2neo
En esta segunda parte del taller utilisamos a Python para interactuar con el grafo.

In [24]:
from py2neo import Node, Relationship

uh = Node("Patrocinador", name="Universidad de la Habana")
neo = Node("Patrocinador", name="Neo Technologies")
python = Node("Patrocinador", name="Python")

neo4j = Node("DB", nombre="Neo4j", tipo="Grafo", nosql=1)
redis = Node("DB", nombre="Redis", tipo="Llave valor", nosql=1)
mongo = Node("DB", nombre="Moongo DB", tipo="Documento", nosql=1)
mySQL = Node("DB", nombre="MySQL", tipo="Relacional", nosql=0)

graph.create(uh | neo | python | neo4j | redis | mongo | mySQL)

In [25]:
taller = graph.find_one("Taller", "título", "Neo4j Python")
kinane = graph.find_one("Persona", "nombre", "Kinane")

print(taller, kinane)

(a518f37:Taller {`título`:"Neo4j Python"}) (e381235:Persona {nombre:"Kinane"})


/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Graph.find_one is deprecated, use NodeSelector instead
  if __name__ == '__main__':
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Graph.find_one is deprecated, use NodeSelector instead
  from ipykernel import kernelapp as app


Creamos las relaciones

In [26]:
graph.create(Relationship(uh, "PATROCINA", taller))
graph.create(Relationship(neo, "PATROCINA", taller))
graph.create(Relationship(python, "PATROCINA", taller))
graph.create(Relationship(kinane, "UTILIZA", neo4j))
graph.create(Relationship(kinane, "UTILIZA", redis))
graph.create(Relationship(kinane, "UTILIZA", mongo))

In [51]:
relaciones = graph.match(start_node=kinane, rel_type="UTILIZA")

for rel in relaciones:
    print (rel.end_node())

(cb371bc:DB {nombre:"Moongo DB",nosql:1,tipo:"Documento"})
(b3e5bf5:DB {nombre:"Redis",nosql:1,tipo:"Llave valor"})
(fd8cbdf:DB {nombre:"Neo4j",nosql:1,tipo:"Grafo"})
